In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

from dbn_nslkdd.dbn.tensorflow import SupervisedDBNClassification

In [21]:
DATA_DIR = '../data'
FILE_PATH_TRAIN_FEATURES = os.path.join(DATA_DIR, 'processed_nslkdd', 'train/train_features.csv')
FILE_PATH_TRAIN_LABLES = os.path.join(DATA_DIR, 'processed_nslkdd', 'train/train_labels.csv')

FILE_PATH_TEST_FEATURES = os.path.join(DATA_DIR, 'processed_nslkdd', 'test/test_features.csv')
FILE_PATH_TEST_LABLES = os.path.join(DATA_DIR, 'processed_nslkdd', 'test/test_labels.csv')

In [22]:
y_df = pd.read_csv(FILE_PATH_TRAIN_FEATURES).iloc[:, 1:]
x_df = pd.read_csv(FILE_PATH_TRAIN_LABLES).iloc[:, 1:]

X_test = pd.read_csv(FILE_PATH_TEST_FEATURES).iloc[:, 1:]
Y_test = pd.read_csv(FILE_PATH_TEST_LABLES).iloc[:, 1:]

In [23]:
X = np.array(x_df)
Y = np.array(y_df).transpose()[0]

In [24]:
n_epochs_rbm = 2
n_iter_backprop = 10

In [25]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=n_epochs_rbm,
                                         n_iter_backprop=n_iter_backprop,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)

In [ ]:
classifier.fit(X, Y)

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 0.000519


In [ ]:
Y_pred = classifier.predict(X_test)
print('Done.\nAccuracy: %f' % accuracy_score(Y_test, Y_pred))